In [1]:
!git clone https://github.com/microsoft/microxcaling.git
!pip install torch==2.2.0 torchaudio==2.2.0
!pip install "numpy<2" --upgrade

%cd microxcaling
!pip install -e .

Cloning into 'microxcaling'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 274 (delta 37), reused 27 (delta 27), pack-reused 221 (from 1)
Receiving objects: 100% (274/274), 963.57 KiB | 48.18 MiB/s, done.
Resolving deltas: 100% (164/164), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.4/755.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━

/content/microxcaling
Obtaining file:///content/microxcaling
  Installing build dependencies ... canceled
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
^C


In [7]:
%cd /content/microxcaling
!pip install -e .

/content/microxcaling
Obtaining file:///content/microxcaling
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.25.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (5.4 kB)
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB

In [59]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import ViTForImageClassification, ViTImageProcessor
from datasets import load_dataset
from tqdm import tqdm
import time


In [1]:
def make_quant_sim(model, w_elem_format, a_elem_format, block_size=32,
                   scale_bits=8, bfloat=16, custom_cuda=False,
                   quantize_backprop=False, round="nearest"):
    """
    Apply microscaling (MX) quantization to the model using the microxcaling library.

    Args:
        model: The model to quantize
        w_elem_format: Weight element format (e.g., 'fp6_e3m2', 'fp8_e4m3', 'int8')
        a_elem_format: Activation element format (e.g., 'fp6_e3m2', 'fp8_e4m3', 'int8')
        block_size: MX scaling block size (default: 32 for MX-compatible formats)
        scale_bits: Number of bits for the MX shared scale (default: 8)
        bfloat: Bfloat format for vector operations (default: 16 for bfloat16)
        custom_cuda: Use custom CUDA kernels (default: False)
        quantize_backprop: Apply quantization on backward pass (default: False)
        round: Rounding mode ('nearest', 'floor', 'even') (default: 'nearest')
    """
    try:
        from mx import finalize_mx_specs
        from mx import mx_mapping
    except ImportError:
        raise ImportError("microxcaling library not found. Please install it first.")

    # Create MX specs dictionary
    mx_specs = {
        'w_elem_format': w_elem_format,
        'a_elem_format': a_elem_format,
        'block_size': block_size,
        'scale_bits': scale_bits,
        'bfloat': bfloat,
        'custom_cuda': custom_cuda,
        'quantize_backprop': quantize_backprop,
        'round': round,
    }

    # Finalize the specs (this sets defaults and validates)
    mx_specs = finalize_mx_specs(mx_specs)

    if mx_specs is None:
        print("No quantization specified, skipping MX injection")
        return

    print(f"Applying MX quantization with specs: {mx_specs}")

    # Auto-inject MX modules and functions
    # This will replace certain torch.nn.* and torch.nn.functional.*
    # modules/functions in the global namespace!
    mx_mapping.inject_pyt_ops(mx_specs)

    print("MX quantization applied successfully!")

In [57]:
model_name = "google/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d_inj(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear_inj(in_features=768, out_features=768, bias=True)
              (key): Linear_inj(in_features=768, out_features=768, bias=True)
              (value): Linear_inj(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear_inj(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear_inj(in_features=768, out_features=3072, bias=Tru

In [74]:
def load_eval_data(n_samples=500):
    ds = load_dataset("imagenet-1k", split="validation", streaming=True)
    ds = ds.take(n_samples)

    images, labels = [], []
    for sample in tqdm(ds, total=n_samples, desc="Loading dataset"):
        images.append(sample['image'].convert("RGB"))
        labels.append(sample['label'])

    return images, labels

images, labels = load_eval_data(n_samples=2000)

Resolving data files:   0%|          | 0/294 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/294 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 2000/2000 [00:12<00:00, 156.55it/s]


In [78]:
def evaluate(model, images, labels, batch_size=32):
    correct = 0
    n_samples = len(images)

    for i in tqdm(range(0, n_samples, batch_size), desc="Evaluating"):
        batch_images = images[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]

        inputs = processor(images=batch_images, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            predictions = model(**inputs).logits.argmax(-1).tolist()

        correct += sum(p == l for p, l in zip(predictions, batch_labels))

    return round(correct / n_samples * 100, 2)

In [79]:
configs = [
    # Baseline
    ("baseline",           None,           None),

    # Single Precision
    ("fp8_e4m3",           'fp8_e4m3',     'fp8_e4m3'),
    ("fp8_e5m2",           'fp8_e5m2',     'fp8_e5m2'),
    ("fp4_e2m1",           'fp4_e2m1',     'fp4_e2m1'),

    # Mixed Precision
    ("w_fp8_a_fp4",        'fp8_e4m3',     'fp4_e2m1'),
    ("w_fp4_a_fp8",        'fp4_e2m1',     'fp8_e4m3'),
]

def get_quantized_model(w_fmt, a_fmt):
    if w_fmt is not None:
        make_quant_sim(model, w_elem_format=w_fmt, a_elem_format=a_fmt)

    fresh_model = ViTForImageClassification.from_pretrained(model_name).to(device)
    fresh_model.eval()
    return fresh_model

results = []
for label, w_fmt, a_fmt in configs:
    print(f"\nRunning: {label}")

    quantized_model = get_quantized_model(w_fmt, a_fmt)
    accuracy = evaluate(quantized_model, images, labels)
    results.append({"config": label, "accuracy": accuracy})


Running: baseline


Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 63/63 [01:08<00:00,  1.09s/it]



Running: fp8_e4m3
Applying MX quantization with specs: {
    "scale_bits": 8,
    "w_elem_format": "fp8_e4m3",
    "a_elem_format": "fp8_e4m3",
    "w_elem_format_bp": "fp8_e4m3",
    "a_elem_format_bp": "fp8_e4m3",
    "a_elem_format_bp_ex": "fp8_e4m3",
    "a_elem_format_bp_os": "fp8_e4m3",
    "mx_flush_fp32_subnorms": false,
    "shared_exp_method": "max",
    "block_size": 32,
    "bfloat": 16,
    "fp": 0,
    "bfloat_subnorms": true,
    "quantize_backprop": false,
    "round": "nearest",
    "round_m": "nearest",
    "round_weight": "nearest",
    "round_output": "nearest",
    "round_grad_weight": "nearest",
    "round_grad_input": "nearest",
    "round_mx_output": "nearest",
    "round_mx_input_grad_input": "nearest",
    "round_mx_weight_grad_input": "nearest",
    "round_mx_grad_output_grad_input": "nearest",
    "round_mx_input_grad_weight": "nearest",
    "round_mx_grad_output_grad_weight": "nearest",
    "softmax_exp2": false,
    "vec_use_exp2": false,
    "vec_use_rec

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

...

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 63/63 [01:08<00:00,  1.09s/it]



Running: w_fp4_a_fp8
Applying MX quantization with specs: {
    "scale_bits": 8,
    "w_elem_format": "fp4_e2m1",
    "a_elem_format": "fp8_e4m3",
    "w_elem_format_bp": "fp4_e2m1",
    "a_elem_format_bp": "fp8_e4m3",
    "a_elem_format_bp_ex": "fp8_e4m3",
    "a_elem_format_bp_os": "fp8_e4m3",
    "mx_flush_fp32_subnorms": false,
    "shared_exp_method": "max",
    "block_size": 32,
    "bfloat": 16,
    "fp": 0,
    "bfloat_subnorms": true,
    "quantize_backprop": false,
    "round": "nearest",
    "round_m": "nearest",
    "round_weight": "nearest",
    "round_output": "nearest",
    "round_grad_weight": "nearest",
    "round_grad_input": "nearest",
    "round_mx_output": "nearest",
    "round_mx_input_grad_input": "nearest",
    "round_mx_weight_grad_input": "nearest",
    "round_mx_grad_output_grad_input": "nearest",
    "round_mx_input_grad_weight": "nearest",
    "round_mx_grad_output_grad_weight": "nearest",
    "softmax_exp2": false,
    "vec_use_exp2": false,
    "vec_use_

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 63/63 [01:08<00:00,  1.09s/it]


In [81]:
baseline_acc = results[0]['accuracy']

print("\n\nResults Summary:")
print(f"{'Config':<20} {'Accuracy':>10} {'Acc Drop':>10}")
print("-" * 42)
for r in results:
    drop = r['accuracy'] - baseline_acc
    print(f"{r['config']:<20} {r['accuracy']:>9.2f}% {drop:>+9.2f}%")



Results Summary:
Config                 Accuracy   Acc Drop
------------------------------------------
baseline                 80.05%     +0.00%
fp8_e4m3                 80.05%     +0.00%
fp8_e5m2                 80.90%     +0.85%
fp4_e2m1                 77.50%     -2.55%
w_fp8_a_fp4              77.90%     -2.15%
w_fp4_a_fp8              80.05%     +0.00%


In [82]:
fp6_configs = [
    ("baseline",     None,           None),
    ("fp6_e3m2",     'fp6_e3m2',     'fp6_e3m2'),
    ("fp6_e2m3",     'fp6_e2m3',     'fp6_e2m3'),

    # mixed fp6 variants
    ("w_fp6e3_a_fp6e2",  'fp6_e3m2',  'fp6_e2m3'),
    ("w_fp6e2_a_fp6e3",  'fp6_e2m3',  'fp6_e3m2'),
]

fp6_results = []
for label, w_fmt, a_fmt in fp6_configs:
    print(f"\nRunning: {label}")
    quantized_model = get_quantized_model(w_fmt, a_fmt)
    accuracy = evaluate(quantized_model, images, labels)
    fp6_results.append({"config": label, "accuracy": accuracy})

baseline_acc = fp6_results[0]['accuracy']
print("\n\nFP6 Results Summary:")
print(f"{'Config':<25} {'Accuracy':>10} {'Acc Drop':>10}")
print("-" * 47)
for r in fp6_results:
    drop = r['accuracy'] - baseline_acc
    print(f"{r['config']:<25} {r['accuracy']:>9.2f}% {drop:>+9.2f}%")


Running: baseline


Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 63/63 [01:08<00:00,  1.09s/it]



Running: fp6_e3m2
Applying MX quantization with specs: {
    "scale_bits": 8,
    "w_elem_format": "fp6_e3m2",
    "a_elem_format": "fp6_e3m2",
    "w_elem_format_bp": "fp6_e3m2",
    "a_elem_format_bp": "fp6_e3m2",
    "a_elem_format_bp_ex": "fp6_e3m2",
    "a_elem_format_bp_os": "fp6_e3m2",
    "mx_flush_fp32_subnorms": false,
    "shared_exp_method": "max",
    "block_size": 32,
    "bfloat": 16,
    "fp": 0,
    "bfloat_subnorms": true,
    "quantize_backprop": false,
    "round": "nearest",
    "round_m": "nearest",
    "round_weight": "nearest",
    "round_output": "nearest",
    "round_grad_weight": "nearest",
    "round_grad_input": "nearest",
    "round_mx_output": "nearest",
    "round_mx_input_grad_input": "nearest",
    "round_mx_weight_grad_input": "nearest",
    "round_mx_grad_output_grad_input": "nearest",
    "round_mx_input_grad_weight": "nearest",
    "round_mx_grad_output_grad_weight": "nearest",
    "softmax_exp2": false,
    "vec_use_exp2": false,
    "vec_use_rec

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

...

Evaluating: 100%|██████████| 63/63 [01:05<00:00,  1.04s/it]



Running: w_fp6e2_a_fp6e3
Applying MX quantization with specs: {
    "scale_bits": 8,
    "w_elem_format": "fp6_e2m3",
    "a_elem_format": "fp6_e3m2",
    "w_elem_format_bp": "fp6_e2m3",
    "a_elem_format_bp": "fp6_e3m2",
    "a_elem_format_bp_ex": "fp6_e3m2",
    "a_elem_format_bp_os": "fp6_e3m2",
    "mx_flush_fp32_subnorms": false,
    "shared_exp_method": "max",
    "block_size": 32,
    "bfloat": 16,
    "fp": 0,
    "bfloat_subnorms": true,
    "quantize_backprop": false,
    "round": "nearest",
    "round_m": "nearest",
    "round_weight": "nearest",
    "round_output": "nearest",
    "round_grad_weight": "nearest",
    "round_grad_input": "nearest",
    "round_mx_output": "nearest",
    "round_mx_input_grad_input": "nearest",
    "round_mx_weight_grad_input": "nearest",
    "round_mx_grad_output_grad_input": "nearest",
    "round_mx_input_grad_weight": "nearest",
    "round_mx_grad_output_grad_weight": "nearest",
    "softmax_exp2": false,
    "vec_use_exp2": false,
    "vec_

Loading weights:   0%|          | 0/200 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 63/63 [01:05<00:00,  1.04s/it]



FP6 Results Summary:
Config                      Accuracy   Acc Drop
-----------------------------------------------
baseline                      80.05%     +0.00%
fp6_e3m2                      80.60%     +0.55%
fp6_e2m3                      80.40%     +0.35%
w_fp6e3_a_fp6e2               80.65%     +0.60%
w_fp6e2_a_fp6e3               80.25%     +0.20%
